In [1]:
import lysis

In [2]:
e = lysis.util.Experiment(r'../../data', experiment_code='2022-12-27-1100')
e.read_file()

In [3]:
print(e)

experiment_code : 2022-12-27-1100
micro_params    : None
macro_params    : macro_version      : diffuse_into_and_along
                  binding_rate       : 0.01
                  pore_size          : 0.00010135
                  diffusion_coeff    : 5e-07
                  binding_sites      : 427.0
                  grid_node_distance : 1.0862
                  rows               : 93
                  cols               : 121
                  first_fiber_row    : 28
                  total_molecules    : 43074
                  moving_probability : 0.2
                  total_trials       : 10
                  total_time         : 600
                  seed               : -2137354075
                  state              : [129281, 362436069, 123456789, -2137354075]
                  data_files         : unbinding_time : tsectPA.dat
                                       lysis_time     : lysismat.dat
                                       total_lyses    : lenlysisvect.dat
       

In [2]:
fg = lysis.EdgeGrid(4, 5, 0, (lysis.util.BoundaryCondition.CONTINUING, lysis.util.BoundaryCondition.REFLECTING))

In [3]:
[fg.neighbor(1, 6, k) for k in range(8)]

[(1, 7), (1, 7), (2, 7), (2, 7), (1, 5), (2, 5), (1, 8), (2, 8)]

In [2]:
lysis.to_fortran_edge_index(0, 3, 4, 5)

10

In [26]:
lysis.from_fortran_edge_index(51, 4, 5)

IndexError: Index (51) out of bounds. Edge indices are in the range [0..50].

In [1]:
import numpy as np

In [4]:
a = np.array([(1,2,3),(4,5,6),(7,8,9)])
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [5]:
b = a[:2,:2]
b

array([[1, 2],
       [4, 5]])

In [7]:
b[0,0] = 0
b

array([[0, 2],
       [4, 5]])

In [8]:
a

array([[0, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [4]:
from enum import Enum, IntEnum, unique

In [5]:
@unique
class Color(IntEnum):
    BLUE = 0
    RED = 1
    GREEN = 2
    
@unique
class Direction(Enum):
    UP = 1
    DOWN = -1

In [13]:
Color.RED == Color.GREEN

False

In [14]:
Color.RED == Direction.UP

False

In [16]:
c = Color

In [17]:
c.RED

<Color.RED: 1>

In [6]:
t = (Direction.UP, Direction.UP, Direction.DOWN)

In [7]:
t[Color.BLUE]

<Direction.UP: 1>

In [9]:
type(Direction.UP)

<enum 'Direction'>

In [12]:
isinstance(Direction.UP, Direction)

True

In [3]:
import sys

In [1]:
import lysis

KeyError: 'util'

In [2]:
import lysis.util

In [ ]:
lysis.util.